# **Principle of Superposition**

- Applicable to any linear system
- The effects can be added
- The effect of a change can be evaluated without taking into account the other effects
- The operators of the transport equation are linear in concentrations

## **1. Loading the needed libraries and coding the equations**

In [1]:
#-------------------------------------------------------#
# Loading the needed libraries                          #
#-------------------------------------------------------#
#--Check and install required packages if not already installed 
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#--Install packages only if they aren't already installed 
if_require("ipywidgets")
if_require("openpyxl")
if_require("xlrd")
if_require("numpy")
if_require("scipy")
if_require("os")

#--Import the rest of libraries
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import scipy as sp
import os

from IPython.display import display, clear_output, HTML
from ipywidgets import Layout, Output
from tkinter import Tk, filedialog

import warnings
warnings.filterwarnings("ignore")

#-- LaTeX configuration, if installed
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Computer Modern Roman"
# })

#--Widgets parameters configuration
param_widgets = { 
                 'readout_format': ".2f", 
                 'style': {'description_width': '300px'}, 
                 'layout': widgets.Layout(width='600px')
                 }  


#-------------------------------------------------------#
# Writing the equations                                 #
#-------------------------------------------------------#
#--Dispersion
def dispersion(alpha: float, v: float) -> float:
    """Calculates the hydrodynamic dispersion coefficient.

    Parameters
    ----------
    alpha : float
        Dispersivity [L]
    v : float
        Groundwater velocity [L/T]

    Returns
    -------
    float
        Dispersion coefficient [L²/T]
    """
    
    D = alpha * v
    return D

#--Dimensionless parameters: Péclet number
def peclet_parameter(x: float, v: int, D: float) -> float:
    """Calculates the Péclet number.

    The Péclet number is a dimensionless parameter that compares
    advective transport to dispersive transport in porous media:
    - High Pe (≫1): advection dominates
    - Low Pe (≪1): dispersion dominates
    
    Parameters
    ----------
    x : float
        Position along the x axis [L]
    v : float
        velocity [L/T]
    D : float
        Dispersion coefficient [L²/T]

    Returns
    -------
    float
        Péclet value [-]
    """
    
    Pe = (v*x)/D
    
    return Pe

#--One-dimensional continuous injection equation  
def oneD_continous_injection(Ci: float, x: float | np.ndarray, v: float, t: float | np.ndarray, 
                             alpha: float, adv: bool) -> np.ndarray:
    """Calculates the concentration in a one-dimensional continous injection,
    with no adsorption or decay (Ogata & Banks, 1961).

    Parameters
    ----------
    Ci : float
        Initial concentration [M/V]
    x : float
        Position along the x axis [L]
    v : float
        Groundwater velocity [L/T]
    t : float
        Time
    alpha : float
        Dispersivity [L]
    adv: Boolean
        If False, the second term of the equation will be considered; 
        If True, the second term of the equation will be neglected.

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    #--Dispersion
    D = dispersion(alpha, v)
    
    #--Dimensionless transport parameters
    Pe = peclet_parameter(x, v, D)
    
    #--Transport equation 
    term1 = sp.special.erfc((x - (v*t))/(2*(np.sqrt(D*t))))
    term2_1 = np.exp(Pe)
    term2_2 = sp.special.erfc((x + (v*t)) / (2*(np.sqrt(D*t))))
                   
    if adv == False:
        C = (Ci/2) * (term1 + (term2_1 * term2_2))
    else:
        C = (Ci/2) * (term1)
        
    return C

## **2. First example: Continuous injection**

Let's imagine a pollutant source located 100 meters away from observation point A. The pollutant is continuously released in two overlapping stages: from the beginning at a concentration of 800 mg/L, and from day 105 onward an additional release of 450 mg/L occurs simultaneously. The advective velocity is 1 m/d and the dispersion coefficient is 4.42 m²/d.

In [2]:
def find_parameters(FS_C, SS_C):   
       
    #--Parameters
    end_time = 350
    second_stage_time = 105
    second_stage_start = np.zeros(second_stage_time)
    nt = np.arange(second_stage_time, end_time)

    first_stage = oneD_continous_injection(FS_C, 100, 1, np.arange(0, end_time), 4.42, False)
    second_stage = oneD_continous_injection(SS_C, 100, 1, np.arange(0, end_time), 4.42, False)
    second_stage = np.append(second_stage_start, second_stage[1:])
    second_stage = second_stage[:end_time]

    merge_stages = first_stage + second_stage
        
    #--Plotting the results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

    ax1.plot(np.arange(0, end_time), first_stage, c="cornflowerblue", label=f"C({FS_C}, t)")
    ax1.plot(np.arange(0, end_time), second_stage, c="limegreen", label=f"C({SS_C}, t)")
    ax1.set_xlim(0, end_time)
    ax1.set_ylim(0, 1900)
    ax1.set_yticks(np.arange(0, 1900, 200))
    ax1.set_xlabel(r'x')
    ax1.set_ylabel('Concentration')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])

    ax2.plot(np.arange(0, end_time), merge_stages, c="tomato", label=r"$C_{1}$ + $C_{11}$")
    ax2.set_xlim(0, end_time)
    ax2.set_ylim(0, 1900)
    ax2.set_yticks(np.arange(0, 1900, 200))
    ax2.set_xlabel(r'x')
    ax2.set_ylabel('Concentration')
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    box = ax2.get_position()
    ax2.set_position([box.x0, box.y0, box.width * 1.1, box.height])

    fig.tight_layout()
    
    plt.show()
        
sliders = [
        widgets.FloatSlider(value=800, min=100, max=900, step=50, description='First Stage Concentration [mg/L]', **param_widgets),
        widgets.FloatSlider(value=450, min=100, max=900, step=50, description='Second Stage Concentration [mg/L]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters, {
        'FS_C': sliders[0],
        'SS_C': sliders[1]
        })

display(ui, out)


Output()

## **3. Second Example: Pulse injection**

Let's now imagine that, in this second example, there is a punctual spill of the pollutant at a concentration of 800 mg/L, also 100 m from the observation point.

In [ ]:
def find_parameters_v2(Ci):   
        #--Parameters
        end_time = 365
        second_stage_time = 105
        second_stage_start = np.zeros(second_stage_time)
        nt = np.arange(second_stage_time, end_time)

        C1 = oneD_continous_injection(Ci, 100, 1, np.arange(0, end_time), 4.42, False)
        C11 = -(oneD_continous_injection(Ci, 100, 1, np.arange(0, end_time), 4.42, False))
        C11 = np.append(second_stage_start, C11[1:])
        C11 = C11[:end_time]
                              
        merge_stages = C1 + C11
        
        maxC11 = np.max(merge_stages)
        merged_lst = list(merge_stages)
        maxC11_pos = merged_lst.index(maxC11)
        noC = []
        for value in range(len(merge_stages)):
                if value > maxC11_pos: 
                        if merge_stages[value] < 1:
                                noC.append(value)                  
                
        #--Plotting the results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

        ax1.plot(np.arange(0, end_time), C1, c="cornflowerblue", label=r"$C_{1}$")
        ax1.plot(np.arange(0, end_time), C11, c="limegreen", label=r"$C_{11}$")
        ax1.hlines(y=0, xmin=0, xmax=end_time, color = "black")
        ax1.set_xlim(0, end_time)
        ax1.set_ylim(-810, 810)
        ax1.set_xlabel(r'time [days]')
        ax1.set_ylabel('Concentration [mg/l]')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])

        ax2.scatter(maxC11_pos, maxC11, c="blue", label=r"Maximum Concentration")
        ax2.plot([0, maxC11_pos], [maxC11, maxC11], c="blue", ls='--')
        ax2.plot([maxC11_pos, maxC11_pos], [maxC11, 0], c="blue", ls='--')
        ax2.scatter(noC[0] + 1, 0.2, c="limegreen", label=r"Concentration ${<}$ 0.2 mg/l")
        ax2.plot(np.arange(0, end_time), merge_stages, c="tomato", label=r"$C_{1}$ + $C_{11}$")
        ax2.set_xlim(0, end_time)
        ax2.set_ylim(0, 810)
        ax2.set_xlabel(r'time [days]')
        ax2.set_ylabel('Concentration [mg/l]')
        ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax2.plot()
        box = ax2.get_position()
        ax2.set_position([box.x0, box.y0, box.width * 1.1, box.height])

        fig.tight_layout()

        plt.show
        
sliders = [
        widgets.FloatSlider(value=400, min=100, max=800, step=50, description='Pollutant Concentration [mg/L]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v2, {
        'Ci': sliders[0]
        })

display(ui, out)

Output()